## Librerías

In [2]:
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

## Funciones para cargar y preprocesar imágenes

In [3]:
def read_train_data(dataframe, reshape_dim=(32, 32)):
    X = []
    y = []
    for index, row in dataframe.iterrows():
        img_path = os.path.join(ROOT_PATH, row['path'].replace('../data/images/', ''))
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Leer en escala de grises
        if image is not None:
            image = cv2.resize(image, reshape_dim)  # Redimensionar a 32x32
            X.append(image)
            y.append(row['label'])
    
    X = np.array(X).reshape(-1, reshape_dim[0], reshape_dim[1], 1)  # Agregar canal de color
    y = np.array(y)
    return X, y

def read_test_data(dataframe, reshape_dim=(32, 32)):
    X = []
    for index, row in dataframe.iterrows():
        img_path = os.path.join(TEST_PATH, f"{row['id_img']}.jpg")
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Leer en escala de grises
        if image is not None:
            image = cv2.resize(image, reshape_dim)  # Redimensionar a 32x32
            X.append(image)
    
    X = np.array(X).reshape(-1, reshape_dim[0], reshape_dim[1], 1)  # Agregar canal de color
    return X


## Cargar los datos de entrenamiento y prueba

In [4]:
# Definir rutas
ROOT_PATH = './data/images/'
TRAIN_PATH = ROOT_PATH + "train/"
TEST_PATH = ROOT_PATH + "test/"

# Leer los archivos CSV
train_csv_path = 'train_set.csv'
test_csv_path = 'test_set.csv'
sample_submission_path = 'sample_submision.csv'

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)
sample_submission_df = pd.read_csv(sample_submission_path)


In [5]:
# Cargar datos de entrenamiento y prueba
X_train, y_train = read_train_data(train_df)
X_test = read_test_data(test_df)


In [6]:
# Normalizar los datos
X_train = X_train / 255.0
X_test = X_test / 255.0


In [7]:
# Ajustar etiquetas a números
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)


In [8]:
# Dividir el conjunto de datos para validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [9]:
# Verificar las formas de los datos
print("Forma de X_train:", X_train.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de X_val:", X_val.shape)
print("Forma de y_val:", y_val.shape)
print("Forma de X_test:", X_test.shape)

Forma de X_train: (23056, 32, 32, 1)
Forma de y_train: (23056,)
Forma de X_val: (5765, 32, 32, 1)
Forma de y_val: (5765,)
Forma de X_test: (7066, 32, 32, 1)


## Definir el modelo mejorado

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Definir el modelo con mejoras
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1), padding='same', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(len(np.unique(y_train)), activation='softmax')
])

# Compilar el modelo con un optimizador ajustado
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Resumen del modelo
model.summary()


c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 4, 4, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 4, 4, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 920,199 (3.51 MB)

 Trainable params: 918,215 (3.50 MB)

 Non-trainable params: 1,984 (7.75 KB)

## Entrenar el modelo con EarlyStopping

In [10]:
# Configurar EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)


In [11]:
# Entrenar el modelo con EarlyStopping
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 47s 117ms/step - accuracy: 0.1663 - loss: 12.5645 - val_accuracy: 0.2147 - val_loss: 10.9400
Epoch 2/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 42s 116ms/step - accuracy: 0.2107 - loss: 11.2101 - val_accuracy: 0.2624 - val_loss: 9.9447
Epoch 3/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 42s 116ms/step - accuracy: 0.2260 - loss: 10.1276 - val_accuracy: 0.2952 - val_loss: 9.0183
Epoch 4/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 48s 132ms/step - accuracy: 0.2502 - loss: 9.1291 - val_accuracy: 0.3343 - val_loss: 8.0994
Epoch 5/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 49s 136ms/step - accuracy: 0.2662 - loss: 8.2283 - val_accuracy: 0.3591 - val_loss: 7.2649
Epoch 6/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 82s 137ms/step - accuracy: 0.2942 - loss: 7.4063 - val_accuracy: 0.3587 - val_loss: 6.5849
Epoch 7/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 83s 140ms/step - accuracy: 0.3138 - loss: 6.6399 - val_accuracy: 0.3818 - val_loss: 5.8918
Epoch 8/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 51s 141ms/step - accuracy: 0.33

In [18]:
# Guardar el modelo actual
model.save('modelo_actual.keras')
print("Modelo guardado en 'modelo_actual.keras'")

Modelo guardado en 'modelo_actual.keras'


In [10]:
#nuevo 4

import time
from tensorflow.keras.callbacks import ModelCheckpoint

# Definir el modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1), padding='same', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(len(np.unique(y_train)), activation='softmax')
])

c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Compilar el modelo con un optimizador ajustado
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Resumen del modelo
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 4, 4, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 4, 4, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 920,199 (3.51 MB)

 Trainable params: 918,215 (3.50 MB)

 Non-trainable params: 1,984 (7.75 KB)

In [13]:
# Configurar ModelCheckpoint para guardar el mejor modelo
checkpoint = ModelCheckpoint('mejor_modelo.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)


In [14]:
# Medir el tiempo de entrenamiento
start_time = time.time()

In [15]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), callbacks=[checkpoint])

end_time = time.time()

Epoch 1/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.1636 - loss: 12.6045
Epoch 1: val_accuracy improved from -inf to 0.14918, saving model to mejor_modelo.keras
361/361 ━━━━━━━━━━━━━━━━━━━━ 53s 137ms/step - accuracy: 0.1637 - loss: 12.6034 - val_accuracy: 0.1492 - val_loss: 11.2145
Epoch 2/100
360/361 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.2169 - loss: 11.1792
Epoch 2: val_accuracy improved from 0.14918 to 0.22723, saving model to mejor_modelo.keras
361/361 ━━━━━━━━━━━━━━━━━━━━ 82s 137ms/step - accuracy: 0.2169 - loss: 11.1777 - val_accuracy: 0.2272 - val_loss: 10.2568
Epoch 3/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.2347 - loss: 10.1177
Epoch 3: val_accuracy improved from 0.22723 to 0.28760, saving model to mejor_modelo.keras
361/361 ━━━━━━━━━━━━━━━━━━━━ 52s 145ms/step - accuracy: 0.2347 - loss: 10.1170 - val_accuracy: 0.2876 - val_loss: 9.0529
Epoch 4/100
361/361 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.2526 - loss: 9.1503
Epoch 

In [17]:
# Mostrar el tiempo de entrenamiento
elapsed_time = end_time - start_time
print(f'Tiempo de entrenamiento: {elapsed_time:.2f} segundos')

# Cargar el mejor modelo guardado
model.load_weights('mejor_modelo.keras')

Tiempo de entrenamiento: 4667.54 segundos


##  Realizar predicciones y crear el archivo de submission

In [18]:
# Realizar predicciones en el conjunto de prueba
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis=1)

221/221 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step


In [19]:
# Crear el DataFrame para submission
submission_df = pd.DataFrame({
    'id_img': test_df['id_img'],
    'label': label_encoder.inverse_transform(predictions)
})

# Guardar el archivo de submission
submission_df.to_csv('submission4.csv', index=False)

# Verificar el contenido del archivo de submission
print(submission_df.head())

   id_img  label
0   10052  happy
1   10065  angry
2   10079  angry
3   10095  angry
4   10121  angry
